<a href="https://colab.research.google.com/github/LuigiSigillo/CartoonGAN/blob/main/CartoonGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
from google.colab import drive
import os
import json
import re
import cv2
import numpy as np
from PIL import Image, ImageFilter
import sys
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import tensorflow as tf
import tensorflow_addons as tfa

drive.mount('/content/drive')
#!unzip /content/drive/My\ Drive/NN/spirited_away.zip -d /content/drive/My\ Drive/NN/
#!ls /content/drive/My\ Drive/NN/
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found GPU at: /device:GPU:0


# Utils

###Save model

In [13]:
models_dir = "/content/drive/My Drive/NN/"

def savemodel(model,problem):
    filename = os.path.join(models_dir, '%s.h5' %problem)
    model.save(filename)
    print("\nModel saved successfully on file %s\n" %filename)

# Save the model MWI-Dataset-1.1_models
savemodel(D,'discriminator_01')



Model saved successfully on file /content/drive/My Drive/NN/discriminator_01.h5



### Load model

In [3]:
from keras.models import load_model

models_dir = "/content/drive/My Drive/NN/"
model_name = "initialization_phase_generator"

def loadmodel(problem):
    filename = os.path.join(models_dir, '%s.h5' %problem)
    try:
        model = load_model(filename)
        print("\nModel loaded successfully from file %s\n" %filename)
    except OSError:    
        print("\nModel file %s not found!!!\n" %filename)
        model = None
    return model

gen_pretrained = loadmodel(model_name)


Model loaded successfully from file /content/drive/My Drive/NN/initialization_phase_generator.h5



### Load datasets

In [4]:
'''#load imgs ?
data_dir = '/content/drive/My Drive/NN/photos_rszd'
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,   validation_split=0.2, horizontal_flip=True)

train_gen = train_datagen.flow_from_directory(
        data_dir,
        subset="training",
        seed=123,
        target_size=(256, 256),
        batch_size=32
        )
validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(256,256),
        batch_size=32,
        )
'''
from glob import glob
def get_dataset(dataset_name, batch_size):
    files = glob(os.path.join('/content/drive/My Drive/NN/', dataset_name, "*"))
    num_images = len(files)
    print(f"Found {num_images}  images in {dataset_name} folder.")
    ds = tf.data.Dataset.from_tensor_slices(files)
    ds = ds.shuffle(num_images)
    ds = ds.repeat()

    def fn(filename):
        x = tf.io.read_file(filename)
        x = tf.image.decode_jpeg(x, channels=3)
        img = tf.cast(x, tf.float32) / 127.5 - 1
        #print("\n tipo img = ",type(img),"\n tipo x = ",type(x), filename, type(filename))
        return img

    ds = ds.map(fn, batch_size)
    ds = ds.batch(batch_size)
    
    steps = int(np.ceil(num_images/batch_size))
    # user iter(ds) to avoid generating iterator every epoch
    return iter(ds), steps


# 1 - Image Preprocessing

### 1.1 - Resizing images


In [ ]:
def resize(path):
    for item in os.listdir(path):
            im = Image.open(path+item)
            f, e = os.path.splitext(item)
            imResize = im.resize((256,256), Image.ANTIALIAS)
            print(f)
            imResize.save(path+"_resized/" + f + ' resized.jpg', 'JPEG', quality=90)

resize('/content/drive/My Drive/NN/your_name')

### 1.2 Apply canny, dilate edge and gaussian

In [ ]:
def edge_smoothing(cartoon_images_filename, smoothed_images_filename):
    print("Edge-smoothing of ", cartoon_images_filename)
    origin = cv2.imread(cartoon_images_filename)
    edges = createEdgesOverlay(origin)
    result = overlayEdges(edges, origin)
    #show_images(origin, edges, result)
    result.save(smoothed_images_filename, "JPEG")

def overlayEdges(edges, origin):
    background = transformFromCV2ToPillowImageFormat(origin)
    background.paste(edges, (0, 0), edges)
    background = background.convert("RGB")
    return background

def transformFromCV2ToPillowImageFormat(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    return Image.fromarray(img)

def createEdgesOverlay(origin):
    edges = cv2.Canny(origin, 30, 300, 3) 
    edges = cv2.dilate(edges, (3, 3))
    edges = cv2.bitwise_not(edges)
    edges = transformFromCV2ToPillowImageFormat(edges)
    makeWhiteBackgroundTransparent(edges)
    edges = edges.filter(ImageFilter.GaussianBlur) #do blurring here because doing it before making background transparent results in white halo

    return edges

def makeWhiteBackgroundTransparent(img):
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)

def show_images(img,edges,result):
    plt.subplot(131),plt.imshow(img)
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])

    plt.subplot(132),plt.imshow(edges)
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
    
    plt.subplot(133),plt.imshow(result)
    plt.title('Result Image'), plt.xticks([]), plt.yticks([])

    plt.show()
    



path_resized = "/content/drive/My Drive/NN/spirited_away_resized/"
path_smoothed = "/content/drive/My Drive/NN/spirited_away_resized_smoothed/"


for filename in os.listdir(path_resized):
  #filename='scene43626 resized.jpg'
  f = filename.split(" ")[0] + " smoothed"
  cartoon_images_filename = path_resized + filename
  smoothed_images_filename = path_smoothed + f
  edge_smoothing(cartoon_images_filename, smoothed_images_filename)



#2 - GAN

## Initialization Phase

### Load VGG19 weights

In [5]:
from keras import applications
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import optimizers



def transferNet(input_shape):
    
    # download model
    base_model = applications.vgg19.VGG19(weights="imagenet", include_top=False, input_shape=input_shape)
    # get the output tensor from a layer of the feature extractor
    tmp_vgg_output = base_model.get_layer("block4_conv3").output
    tmp_vgg_output = Conv2D(512, (3, 3), activation='linear', padding='same',name='block4_conv4')(tmp_vgg_output)
    
    vgg = Model(inputs=base_model.input, outputs=tmp_vgg_output)
    vgg.load_weights(os.path.expanduser(os.path.join("~", ".keras", "models","vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")), by_name=True)

    return vgg


input_shape =(256,256,3)
# load the pre-trained model
vgg = transferNet(input_shape)
vgg.summary()
!ls ~/.keras/models


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

### Generator

In [ ]:
from keras import applications
from keras import layers
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import *
from keras import regularizers
from keras import optimizers
import tensorflow as tf
import tensorflow_addons as tfa


def resblock(x_init):
    x = Conv2D(256, kernel_size=3, strides=1, padding='same')(x_init)
    #x = (ZeroPadding2D(padding=(1, 1)))(x)
    
    x = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(x)
    x = tf.nn.relu(x)
    x = (Conv2D(256, kernel_size=3, strides=1, padding='same'))(x)
    #x = (ZeroPadding2D(padding=(1, 1)))(x)
    x = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(x)    

    return x + x_init

def create_generator(input_shape):
    #model = Sequential()
    #k7n64s1
    input_t = tf.keras.layers.Input(shape=input_shape)
    
    model = (Conv2D(input_shape=input_shape, filters=64, kernel_size=7, strides=1, padding='same'))(input_t)
    #model = (ZeroPadding2D(padding=(3, 3)))(model)
    model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
    model = tf.nn.relu(model)
    
    #### Down-Convolution
    
    #k3 n128 s2
    model = (Conv2D(filters=128, kernel_size=3, strides=2, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    #k3 n128 s1
    model = (Conv2D(filters=128, kernel_size=3, strides=1, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    
    model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
    model = tf.nn.relu(model)


    #k3 n256 s2
    model = (Conv2D(filters=256, kernel_size=3, strides=2, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)

    #k3 n256 s1
    model = (Conv2D(filters=256, kernel_size=3, strides=1, padding='same', name="test"))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    
    model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
    model = tf.nn.relu(model)



    #TODO
    # residual blocks
    for i in range(8):#number of res blocks
        '''model = (Conv2D(256, kernel_size=3, strides=1, padding='valid',activation="relu"))(model)
        model = (ZeroPadding2D(padding=(1, 1)))(model)
        
        model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
        model = tf.nn.relu(model)

        model = (Conv2D(256, kernel_size=3, strides=1, padding='valid',activation="relu"))(model)
        model = (ZeroPadding2D(padding=(1, 1)))(model)
        model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)'''
        model = resblock(model)

        
        
    
    # Up-convolution
    model = (Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='same',output_padding=1))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    
    model = (Conv2D(filters=128, kernel_size=3, strides=1, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    
    model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
    model = tf.nn.relu(model)
    #######################
    model = (Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same',output_padding=1))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)
    
    model = (Conv2D(filters=64, kernel_size=3, strides=1, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(1, 1)))(model)

    model = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True)(model)
    model = tf.nn.relu(model)

    ###
    model = (Conv2D(filters=3, kernel_size=7, strides=1, padding='same'))(model)
    #model = (ZeroPadding2D(padding=(3, 3)))(model)
    model = tf.tanh(model)

    #model.compile(loss=tf.keras.losses.BinaryCrossentropy)
    model = Model(inputs=input_t, outputs=model, name="generator")

    return model

G = create_generator(input_shape=(256,256,3))
#G.build(input_shape=(256,256,3)) #todo
#G.summary()


#### Generator training

In [ ]:
#TODO
from glob import glob
import os
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt

def pretrain_step(vgg, input_images, generator, optimizer):
    with tf.GradientTape() as tape:
        generated_images = generator(input_images)

        plt.imshow(generated_images[0])
        plt.imshow(input_images[0])
        mae = tf.keras.losses.MeanAbsoluteError()
        c_loss = mae(vgg(input_images), vgg(generated_images))

        gradients = tape.gradient(c_loss, generator.trainable_variables)
        optimizer.apply_gradients(zip(gradients, generator.trainable_variables))
        content_loss_metric = tf.keras.metrics.Mean("content_loss", dtype=tf.float32)
        content_loss_metric(c_loss)

def pretrain_generator(dataset_name,batch_size):
    summary_writer = tf.summary.create_file_writer(os.path.join('/content/drive/My Drive/NN', "pretrain"))
    print(f"Starting to pretrain generator with 10 epochs...")
    print(f"Building `{dataset_name}` dataset")
    dataset, steps_per_epoch = get_dataset(dataset_name=dataset_name, batch_size=batch_size)
    
    generator = G
    #generator(tf.keras.Input(shape=(None,256,256, 3)))
    #generator.summary()

    print("Setting up optimizer to update generator's parameters...")
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0008, beta_1=0.5)

    trained_epochs = 0
    epochs = 10

    print("Proceeding pretraining without sample images...")

    print("Starting pre-training loop, setting up summary writer to record progress on TensorBoard...")

    for epoch in range(epochs):
        epoch_idx = trained_epochs + epoch + 1

        for step in tqdm(range(1, steps_per_epoch + 1), desc=f"Pretrain Epoch {epoch + 1}/{epochs}"):
            image_batch = dataset.next()
            pretrain_step(vgg, image_batch, generator, optimizer)

            if step % 100 == 0: #100 = pretrain_reporting_steps
                global_step = (epoch_idx - 1) * steps_per_epoch + step
                with summary_writer.as_default():
                    tf.summary.scalar('content_loss', tf.keras.metrics.Mean("content_loss", dtype=tf.float32).result(), step=global_step)

                tf.keras.metrics.Mean("content_loss", dtype=tf.float32).reset_states()
        gc.collect()
    return generator
        
gen_pretrained = pretrain_generator("photos_rszd/photos_resized", 16)


#### Show photos results

In [ ]:
'''y_true = [[0., 1.], [0., 0.]]
y_pred = [[1., 1.], [1., 0.]]
mae = tf.keras.losses.MeanAbsoluteError()
print(mae(y_true, y_pred).numpy())'''

#savemodel(gen_pretrained,'initialization_phase_generator')
dataset, steps_per_epoch = get_dataset(dataset_name="photos_rszd/photos_resized", batch_size=16)
input_images = dataset.next()
plt.imshow(input_images[0])

#gen_pretrained.predict(input_images[0])
#%load_ext tensorboard
#%tensorboard --logdir "/content/drive/My Drive/NN/pretrain"

## Discriminator

In [6]:
from keras.layers import *
def create_discriminator(input_shape):
    model = Sequential()
    #k7n64s1
    model.add(Conv2D(input_shape=input_shape, filters=32, kernel_size=3, strides=1, padding='same',activation=LeakyReLU()))
    
    #k3n64s2
    model.add(Conv2D(filters=64, kernel_size=3, strides=2, padding='same',activation=LeakyReLU(alpha=0.2)))
    
    #k3n128s1
    model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same'))
    model.add(tfa.layers.InstanceNormalization(axis=3, center=True, scale=True))
    model.add(LeakyReLU(alpha=0.2))
    
    #k3n128s2
    model.add(Conv2D(filters=128, kernel_size=3, strides=2, padding='same',activation=LeakyReLU(alpha=0.2)))
    #k3n256s1
    model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same'))
    model.add(tfa.layers.InstanceNormalization(axis=3, center=True, scale=True))
    model.add(LeakyReLU(alpha=0.2))
    #k3n256s1
    model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same'))
    model.add(tfa.layers.InstanceNormalization(axis=3, center=True, scale=True))
    model.add(LeakyReLU(alpha=0.2))
    #k3n1s1
    model.add(Conv2D(filters=1, kernel_size=3, strides=1, padding='same'))
   
    
    #model.compile(loss=tf.keras.losses.BinaryCrossentropy)
    return model


D = create_discriminator(input_shape=(256,256,3))
D.summary()  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
instance_normalization (Inst (None, 128, 128, 128)     256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 256)       2

## Adversarial model

#### Loss functions

In [7]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

mae = tf.keras.losses.MeanAbsoluteError()


def discriminator_loss (real, fake, real_blur):
    real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(real), logits=real))
    fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(fake), logits=fake))
    real_blur_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(real_blur), logits=real_blur))
    
    return real_loss+fake_loss+real_blur_loss

def generator_loss(fake, vgg_loss):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(fake), logits=fake)) + vgg_loss

#### Get datasets

In [8]:
cartoon_set,steps_per_epoch = get_dataset("paprika_resized",8)
cartoon_smoothed_set, steps_per_epoch = get_dataset("paprika_resized_smoothed",8)
normal_photos_set,steps_per_epoch = get_dataset("photos_rszd/photos_resized",8)

Found 3591  images in paprika_resized folder.
Found 3592  images in paprika_resized_smoothed folder.
Found 3747  images in photos_rszd/photos_resized folder.


### training GAN

In [9]:
from IPython import display
import matplotlib.pyplot as plt
import time
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input)

    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5)
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
    
@tf.function
def train_step(cartoon_set_batch, cartoon_smoothed_batch, normal_photos_batch):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = gen_pretrained(normal_photos_batch)

        real_output = D(cartoon_set_batch)
        fake_output = D(generated_images)
        real_smooth_output = D(cartoon_smoothed_batch)
        
        vgg_loss = mae(vgg(normal_photos_batch), vgg(cartoon_set_batch))
        gen_loss = generator_loss(fake_output, vgg_loss)
        disc_loss = discriminator_loss(real_output, fake_output, real_smooth_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, gen_pretrained.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, D.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, gen_pretrained.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, D.trainable_variables))


def train(dataset_1, dataset_2, dataset_3, epochs):
    for epoch in range(epochs):
        start = time.time()

        for step in tqdm(
                    range(1, steps_per_epoch + 1),
                    desc=f'Train {epoch + 1}/{epochs}',
                    total=steps_per_epoch):
            cart_batch = dataset_1.next()
            cart_smooth_batch = dataset_2.next()
            norm_photos_batch = dataset_3.next()
            train_step(cart_batch, cart_smooth_batch, norm_photos_batch)

        # Produce images for the GIF as we go
        #display.clear_output(wait=True)
        #generate_and_save_images(gen_pretrained,epoch + 1)
        '''# Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)'''

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Generate after the final epoch
    display.clear_output(wait=True)
    #generate_and_save_images(gen_pretrained,epochs)

train(cartoon_set, cartoon_smoothed_set, normal_photos_set, 5)

Time for epoch 1 is 1200.456134557724 sec
Time for epoch 2 is 526.7850053310394 sec
Time for epoch 3 is 526.9260878562927 sec
Time for epoch 4 is 526.417640209198 sec
Time for epoch 5 is 527.0203611850739 sec


### Train

In [ ]:
def create_GAN():
    optimizer = tf.keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=6e-8)

    discriminator_model = Sequential()
    discriminator_model.add(D)
    discriminator_model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])

    optimizer = tf.keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=3e-8)
    adv_model = Sequential()
    adv_model.add(G)
    adv_model.add(D)
    adv_model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    return adv_model

In [ ]:
adv_model = create_GAN()
adv_model.fit(
        train_gen,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800,
        verbose=1)

Epoch 1/50
